In [1]:
import os
import time
import sys
import itertools
import json
import librosa as lr
import librosa.display as lrd
import numpy as np
import os.path as osp

import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.metrics import mean_squared_error

%matplotlib inline

sys.path.append("..")
import pickle as pk


In [13]:
res_path = '/home/christie/results/model_190521_1547_conv_ae_a4bc31/results_190523_0853.pkl'

### Phase 2 - ds1
# tcn-mag-train: results/model_190517_1742_conv_tcn_182082/results_190522_2301.pkl
# tcn-db-train: results/model_190517_1547_conv_tcn_a30f3a/results_190521_2237.pkl
# tcn-mag-test: results/model_190517_1742_conv_tcn_182082/results_190523_0016.pkl
# tcn-db-test: results/model_190517_1547_conv_tcn_a30f3a/results_190523_0026.pkl

### Phase 2 - ds2 
# ae-db-test: results/model_190521_1547_conv_ae_a4bc31/results_190523_0853.pkl
# ae-db-train:

In [14]:
with open(res_path,'rb') as f: 
    res = pk.load(f)
res


mse  \
filepath                                           noise_variation                                                
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.087052   
                                                   (<libs.processing.take_file_as_noise object at ...  0.046158   
                                                   (<libs.processing.take_file_as_noise object at ...  0.138432   
                                                   (<libs.processing.take_file_as_noise object at ...  0.052448   
                                                   (<libs.processing.take_file_as_noise object at ...  0.058906   
                                                   (<libs.processing.take_file_as_noise object at ...  0.039050   
                                                   (<libs.processing.take_file_as_noise object at ...  0.074987   
                                                   (<libs.processing.take_file_as_noise object at ...  0.039676   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.126197   
                                                   (<libs.processing.take_file_as_noise object at ...  0.052522   
                                                   (<libs.processing.take_file_as_noise object at ...  0.166149   
                                                   (<libs.processing.take_file_as_noise object at ...  0.058457   
                                                   (<libs.processing.take_file_as_noise object at ...  0.063970   
                                                   (<libs.processing.take_file_as_noise object at ...  0.046502   
                                                   (<libs.processing.take_file_as_noise object at ...  0.089592   
                                                   (<libs.processing.take_file_as_noise object at ...  0.047677   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.073608   
                                                   (<libs.processing.take_file_as_noise object at ...  0.036571   
                                                   (<libs.processing.take_file_as_noise object at ...  0.147797   
                                                   (<libs.processing.take_file_as_noise object at ...  0.038539   
                                                   (<libs.processing.take_file_as_noise object at ...  0.050765   
                                                   (<libs.processing.take_file_as_noise object at ...  0.032950   
                                                   (<libs.processing.take_file_as_noise object at ...  0.064725   
                                                   (<libs.processing.take_file_as_noise object at ...  0.033079   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.081781   
                                                   (<libs.processing.take_file_as_noise object at ...  0.035128   
                                                   (<libs.processing.take_file_as_noise object at ...  0.116554   
                                                   (<libs.processing.take_file_as_noise object at ...  0.036587   
                                                   (<libs.processing.take_file_as_noise object at ...  0.042540   
                                                   (<libs.processing.take_file_as_noise object at ...  0.031396   
                                                   (<libs.processing.take_file_as_noise object at ...  0.063830   
                                                   (<libs.processing.take_file_as_noise object at ...  0.031864   

                                                                                                             sdr  \
filepath                                           noise_variation               

In [15]:
res.shape
nb_occur = res['mse'].shape[0]
print(nb_occur)

32


In [26]:
ana_types = {'mse', 'sdr', 'sir', 'sar', 'stoi', 'pesq'}
snr = {5, 15}
noises = {1,2,3,4}


4

In [24]:
# metrics averaging on noises, for each snr

for a in ana_types:
    print('metrics :',a)
    analysis_5 =  [res[a][2*im] for im in range(nb_occur//2) ]#np.zeros()
    analysis_15 = [res[a][2*im+1] for im in range(nb_occur//2) ]
    # print("mse_5 :", mse_5, ",\n mse_15 :", mse_15)

    mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
    mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
    print("mean_5: {}, std_5: {}".format(mean_5, std_5))
    print("mean_15 {}, std_15: {}".format(mean_15, std_15))

metrics :, sdr
mean_5: 13.332550809493593, std_5: 3.0110775991613363
mean_15 20.37601660908333, std_15: 1.261275489871322
metrics :, stoi
mean_5: 0.8638776770043112, std_5: 0.031645774254895453
mean_15 0.9149813692598691, std_15: 0.03299984253400409
metrics :, sir
mean_5: 18.372503852851935, std_5: 3.7853282513970856
mean_15 30.375720665925783, std_15: 3.6165894238070346
metrics :, mse
mean_5: 0.09043034671294184, std_5: 0.036079459859567946
mean_15 0.0411628417951044, std_15: 0.008131826983004103
metrics :, sar
mean_5: 15.406077030432602, std_5: 3.110448697067232
mean_15 20.932082813371842, std_15: 0.9585326247391598
metrics :, pesq
mean_5: 3.0607031633456545, std_5: 0.28900867355646565
mean_15 4.080586766203244, std_15: 0.09348709904209683


In [29]:
# metrics for each noise, for each snr

for a in ana_types:
    print('\n metrics :', a)
    for n in range(len(noises)):
        print('noise no :', n)
        analysis_5 =  [res[a][8*im+2*n] for im in range(nb_occur//8) ]
        analysis_15 = [res[a][8*im+2*n+1] for im in range(nb_occur//8) ]
    #     print(analysis_5)

        mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
        mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
        print("mean_5: {}, std_5: {}".format(mean_5, std_5))
        print("mean_15 {}, std_15: {}".format(mean_15, std_15))
    


 metrics : sdr
noise no : 0
mean_5: 12.266562337199264, std_5: 0.6711200011852978
mean_15 19.87175415573379, std_15: 0.5327299732222875
noise no : 1
mean_5: 9.506973151285528, std_5: 0.6194824261500117
mean_15 18.78363971808442, std_15: 0.8808921637468105
noise no : 2
mean_5: 17.607906330486657, std_5: 0.9654544891677154
mean_15 21.465440130699307, std_15: 0.41051071300948905
noise no : 3
mean_5: 13.948761419002935, std_5: 0.2652058474622875
mean_15 21.383232431815813, std_15: 0.37907791221170134

 metrics : stoi
noise no : 0
mean_5: 0.852311481187704, std_5: 0.01741834368801666
mean_15 0.9126004884182006, std_15: 0.030920389999948356
noise no : 1
mean_5: 0.8525005738831315, std_5: 0.033408284775810815
mean_15 0.9127558570968408, std_15: 0.032599305456518415
noise no : 2
mean_5: 0.8929166062570251, std_5: 0.02834065085605791
mean_15 0.9212745850638832, std_15: 0.03656329808977341
noise no : 3
mean_5: 0.8577820466893844, std_5: 0.025288019913553134
mean_15 0.9132945464605514, std_15: 0

In [ ]:
# metrics for each noise, for each snr

for a in ana_types:
    print('\n metrics :', a)
    for n in range(len(noises)):
        print('noise no :', n)
        analysis_5 =  [res[a][8*im+2*n] for im in range(nb_occur//8) ]
        analysis_15 = [res[a][8*im+2*n+1] for im in range(nb_occur//8) ]
    #     print(analysis_5)

        mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
        mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
        print("mean_5: {}, std_5: {}".format(mean_5, std_5))
        print("mean_15 {}, std_15: {}".format(mean_15, std_15))

In [16]:
# mse

mse_5 =  [res['mse'][2*im] for im in range(nb_occur//2) ]#np.zeros()
mse_15 = [res['mse'][2*im+1] for im in range(nb_occur//2) ]
# print("mse_5 :", mse_5, ",\n mse_15 :", mse_15)

mean_mse_5, std_mse_5 = np.mean(mse_5), np.std(mse_5)
mean_mse_15, std_mse_15 = np.mean(mse_15), np.std(mse_15)
print("mean_mse_5: {}, std_mse_5: {}".format(mean_mse_5, std_mse_5))
print("mean_mse_15 {}, std_mse_15: {}".format(mean_mse_15, std_mse_15))

mean_mse_5: 0.09043034671294184, std_mse_5: 0.036079459859567946
mean_mse_15 0.0411628417951044, std_mse_15: 0.008131826983004103


In [17]:
# sdr

sdr_5 =  [res['sdr'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sdr_15 = [res['sdr'][2*im+1] for im in range(nb_occur//2) ]
# print("sdr_5 :", sdr_5, ",\n sdr_15 :", sdr_15)

mean_sdr_5, std_sdr_5 = np.mean(sdr_5), np.std(sdr_5)
mean_sdr_15, std_sdr_15 = np.mean(sdr_15), np.std(sdr_15)
print("mean_sdr_5: {}, std_sdr_5: {}".format(mean_sdr_5, std_sdr_5))
print("mean_sdr_15 {}, std_sdr_15: {}".format(mean_sdr_15, std_sdr_15))

mean_sdr_5: 13.332550809493593, std_sdr_5: 3.0110775991613363
mean_sdr_15 20.37601660908333, std_sdr_15: 1.261275489871322


In [18]:
# sir

sir_5 =  [res['sir'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sir_15 = [res['sir'][2*im+1] for im in range(nb_occur//2) ]
# print("sir_5 :", sir_5, ",\n sir_15 :", sir_15)

mean_sir_5, std_sir_5 = np.mean(sir_5), np.std(sir_5)
mean_sir_15, std_sir_15 = np.mean(sir_15), np.std(sir_15)
print("mean_sir_5: {}, std_sir_5: {}".format(mean_sir_5, std_sir_5))
print("mean_sir_15 {}, std_sir_15: {}".format(mean_sir_15, std_sir_15))

mean_sir_5: 18.372503852851935, std_sir_5: 3.7853282513970856
mean_sir_15 30.375720665925783, std_sir_15: 3.6165894238070346


In [19]:
# sar

sar_5 =  [res['sar'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sar_15 = [res['sar'][2*im+1] for im in range(nb_occur//2) ]
# print("sar_5 :", sar_5, ",\n sar_15 :", sar_15)

mean_sar_5, std_sar_5 = np.mean(sar_5), np.std(sar_5)
mean_sar_15, std_sar_15 = np.mean(sar_15), np.std(sar_15)
print("mean_sar_5: {}, std_sar_5: {}".format(mean_sar_5, std_sar_5))
print("mean_sar_15 {}, std_sar_15: {}".format(mean_sar_15, std_sar_15))

mean_sar_5: 15.406077030432602, std_sar_5: 3.110448697067232
mean_sar_15 20.932082813371842, std_sar_15: 0.9585326247391598


In [20]:
# stoi

stoi_5 =  [res['stoi'][2*im] for im in range(nb_occur//2) ]#np.zeros()
stoi_15 = [res['stoi'][2*im+1] for im in range(nb_occur//2) ]
# print("stoi_5 :", stoi_5, ",\n stoi_15 :", stoi_15)

mean_stoi_5, std_stoi_5 = np.mean(stoi_5), np.std(stoi_5)
mean_stoi_15, std_stoi_15 = np.mean(stoi_15), np.std(stoi_15)
print("mean_stoi_5: {}, std_stoi_5: {}".format(mean_stoi_5, std_stoi_5))
print("mean_stoi_15 {}, std_stoi_15: {}".format(mean_stoi_15, std_stoi_15))

mean_stoi_5: 0.8638776770043112, std_stoi_5: 0.031645774254895453
mean_stoi_15 0.9149813692598691, std_stoi_15: 0.03299984253400409


In [21]:
# pesq

pesq_5 =  [res['pesq'][2*im] for im in range(nb_occur//2) ]#np.zeros()
pesq_15 = [res['pesq'][2*im+1] for im in range(nb_occur//2) ]
# print("pesq_5 :", pesq_5, ",\n pesq_15 :", pesq_15)

mean_pesq_5, std_pesq_5 = np.mean(pesq_5), np.std(pesq_5)
mean_pesq_15, std_pesq_15 = np.mean(pesq_15), np.std(pesq_15)
print("mean_pesq_5: {}, std_pesq_5: {}".format(mean_pesq_5, std_pesq_5))
print("mean_pesq_15 {}, std_pesq_15: {}".format(mean_pesq_15, std_pesq_15))


mean_pesq_5: 3.0607031633456545, std_pesq_5: 0.28900867355646565
mean_pesq_15 4.080586766203244, std_pesq_15: 0.09348709904209683
